## Importing the libraries

In [77]:
# import the libraries
import os
import json
import pandas as pd
import numpy as np
import openpyxl
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score

import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.callbacks import EarlyStopping
from keras.losses import SparseCategoricalCrossentropy
from keras.metrics import Accuracy, F1Score, Precision, Recall
from keras.optimizers import SGD, Adam, Adagrad, RMSprop, Nadam, AdamW
from keras.src.losses import loss
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.applications import MobileNetV2 # we need to be able to use pretrained model

BASE_DIR = os.getcwd() # it will be used in whole processes to easily access folders

print(tf.__version__)

2.20.0


## Variable initialization (that will help us to update parameters easily)

In [ ]:
TRAIN_DATA_DIR = os.path.join(BASE_DIR, "dataset", "train")
VAL_DATA_DIR = os.path.join(BASE_DIR, "dataset", "val")
FULL_DATA = True # if set as True then it will train over concatenated data (train + val), if false then only train (as usual)


MODEL_SAVE_DIR = os.path.join(BASE_DIR, "models")
MODEL_NAME = 'cnn_model_phase3_full_data'
TESTING_MODE = False # if it is set as True then it will help you easily test pipeline
# !!! WARNING !!!: if you set as True it can result saving wrong model

# Parameters that will be used in modeling
MOMENTUM= 0.9
NESTEROV=True

INPUT_IMG_SIZE = (224, 168)
NUM_CLASSES = 9
CHANNELS = 3

EPOCHS = 7
LEARNING_RATE = 1e-3
PATIENCE = 15
BATCH = 32
SEED = 42
tf.keras.utils.set_random_seed(SEED)

### Target class names

In [69]:
# target class names
if os.path.isdir(TRAIN_DATA_DIR):
    folder_contents = os.listdir(TRAIN_DATA_DIR)
    print(f"Contents of '{TRAIN_DATA_DIR}':")
    for item in folder_contents:
        print(item)
else:
    print(f"Error: '{TRAIN_DATA_DIR}' is not a valid directory.")

Contents of 'c:\Users\Shahbaz\Desktop\data science\from git\ku-buildings-classifier\dataset\train':
basement
church
entrance
georgianum
kreuztor
ku
pink
room
wfi


## Train - Validation split

In [ ]:
# 1) Train split
print(f"Loading Training Data (Color Mode: rgb):")
train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DATA_DIR,
    labels='inferred',
    label_mode='int',
    color_mode="rgb",
    batch_size=BATCH,
    image_size=INPUT_IMG_SIZE,
    seed=SEED
)

# 2) Validation split
print("\nLoading Validation Data:")
val_ds = tf.keras.utils.image_dataset_from_directory(
    VAL_DATA_DIR,
    labels='inferred',
    label_mode='int',
    color_mode="rgb",
    batch_size=BATCH,
    image_size=INPUT_IMG_SIZE,
    seed=SEED
)

print('\n')
print("Classes of train:", train_ds.class_names)
num_classes = len(train_ds.class_names)

print("Classes of validation:", val_ds.class_names)
num_classes = len(val_ds.class_names)

Loading Training Data (Color Mode: rgb):
Found 6769 files belonging to 9 classes.

Loading Validation Data:
Found 1698 files belonging to 9 classes.


Classes of train: ['basement', 'church', 'entrance', 'georgianum', 'kreuztor', 'ku', 'pink', 'room', 'wfi']
Classes of validation: ['basement', 'church', 'entrance', 'georgianum', 'kreuztor', 'ku', 'pink', 'room', 'wfi']


### writing target class labels as json file

In [ ]:
# extract the class names (Keras automatically sorts them) to use easily in testing
class_names = train_ds.class_names
class_mapping = {i: name for i, name in enumerate(class_names)}

# save it as a JSON in your models directory
os.makedirs(MODEL_SAVE_DIR, exist_ok=True)
mapping_path = os.path.join(MODEL_SAVE_DIR, 'class_mapping.json')

with open(mapping_path, 'w') as f:
    json.dump(class_mapping, f, indent=4)

print(f"\nSuccessfully saved class mapping to {mapping_path}")
print("Mapping dictionary:", class_mapping)


Successfully saved class mapping to C:\Users\Shahbaz\Desktop\data science\from git\ku-buildings-classifier\models\class_mapping.json
Mapping dictionary: {0: 'basement', 1: 'church', 2: 'entrance', 3: 'georgianum', 4: 'kreuztor', 5: 'ku', 6: 'pink', 7: 'room', 8: 'wfi'}


## ⚠️ Important Note on Data Normalization

Standard image normalization (dividing by `255.0`) is intentionally skipped in the data pipeline. 

Because we are using **MobileNetV2** as our base model, it specifically requires pixel values to be scaled between `[-1, 1]` rather than the standard `[0, 1]`. 

To handle this cleanly, the MobileNet-specific rescaling is built directly into the sequential model architecture rather than mapping it to the dataset:
`layers.Rescaling(1./127.5, offset=-1)`

In [ ]:
# THE DRY RUN LOGIC (smoke test before the attack)
if TESTING_MODE:
    print("testing mode activated and it helps smoke test")
    train_ds = train_ds.take(2)
    val_ds = val_ds.take(1)

# Speed, this is not that important but recommended
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
val_ds   = val_ds.prefetch(tf.data.AUTOTUNE)

### count of train data and shape

In [ ]:
# information of train data
for images, labels in train_ds.take(1):
    print(images.shape)

total_images = len(train_ds) * BATCH
print(f"Number of total training batches: {len(train_ds)}")
print(f"Number of total training images (approximately): {total_images}")

(32, 224, 168, 3)
Number of total training batches: 212
Number of total training images (approximately): 6784


## Pretrained model: MobileNet V2
* **`include_top = False`**: Avoids using the 1000-class output layer of the pretrained model. By avoiding the original classification (the last layer), so we can attach our own custom Dense layers.
* **`weights = 'imagenet'`**: Initializes the model with parameters pre-trained on the massive ImageNet dataset (over 1 million images). Instead of starting from scratch with random, untrained weights, the model already acts as a powerful feature extractor that knows how to identify fundamental visual patterns like edges, corners, and textures.

In [ ]:
basemodel = tf.keras.applications.MobileNetV2(
    input_shape=(224,168,3),
    include_top=False,
    weights="imagenet"
)

# basemodel.summary() # you may get information about pretrained model

C:\Users\Shahbaz\AppData\Local\Temp\ipykernel_9988\1595368369.py:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  basemodel = tf.keras.applications.MobileNetV2(


### We are seting the weights fixed or freezed so they do not get  (for now)

In [70]:
basemodel.trainable = False 

### Data Augmentation
Data augmentation is a technique used to artificially expand the training dataset by applying random transformations to the images (such as rotating, zooming, or flipping). 

* **Reduces overfitting (a bit):** By using different type of the input images, it prevents model from simply memorizing the repeated patterns of the training data.
* **Improves generalization:** It forces the model to learn the actual structural features of the KU buildings. This ensures the model can still accurately recognize a building in the real world, even if a user's test photo is taken from a weird angle, zoomed in, or slightly tilted.

In [46]:
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomTranslation(0.1,0.1),
    layers.RandomContrast(0.1),
])

## Model Architecture

In [ ]:
model= keras.Sequential([
    layers.Input(shape=(224,168,3)),
    data_augmentation,
    layers.Rescaling(1./127.5,offset=-1),
    basemodel,
    layers.GlobalAveragePooling2D(), ## Prof. Voigtlaenders suggestion to use this instead of Flatten()
    
    layers.Dense(256,activation="relu", kernel_initializer="he_normal"),
    layers.BatchNormalization(), # i learned this new, so we normalize the outputs of the layers so that mean =0 and var=1
    layers.Dropout(0.4), # also from the deep learning class
    
    layers.Dense(128,activation="relu",kernel_initializer="he_normal"),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    
    layers.Dense(64,activation="relu", kernel_initializer="he_normal"),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    
    layers.Dense(32,activation="relu",kernel_initializer="he_normal"),
    
    layers.Dense(NUM_CLASSES,activation="softmax") # num_class in our class in PHASE 3 is equal to 9 since we have combined multiple subclasses to a unified class, before in phase 1 it was 21 categories
]) 

### Early Stopping & Restoring Weights
Early stopping acts as a safety brake during training. It monitors the model's performance and automatically stops the iterations if the accuracy stops improving for a set amount of time (our `patience` variable). 

* **`restore_best_weights = True`**: If the model starts to overfit and actually gets worse during those extra patience epochs, this setting forces Keras to "rewind" and grab the exact parameters from the model's absolute best epoch. This prevents us from accidentally saving a ruined, overfitted version of our model.

### Early stopping: helps model to stop iterations when it stops improving over the given metric (**`monitor = 'val_loss'`**)
* **`restore_best_weights = True`** helps us to convert the model parameters to 5 step previous (**`patience = 5`**) version which helps us to prevent meaningless overfitting over data

In [14]:
early=EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True,
    verbose=1
)

## Fitting the model

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"]
)

# if FULL_DATA is set as True in the model parameters initialization step then it will train over full data
if FULL_DATA:
    full_ds = train_ds.concatenate(val_ds)
    model.fit(
        full_ds,
        epochs=EPOCHS,
        callbacks=[early]
    )
else:
    model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=EPOCHS,
        callbacks=[early]
    )

Epoch 1/7


266/266 ━━━━━━━━━━━━━━━━━━━━ 213s 779ms/step - accuracy: 0.7425 - loss: 0.8258
Epoch 2/7


c:\Users\Shahbaz\Desktop\data science\from git\ku-buildings-classifier\.venv\Lib\site-packages\keras\src\callbacks\early_stopping.py:99: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


266/266 ━━━━━━━━━━━━━━━━━━━━ 204s 766ms/step - accuracy: 0.9275 - loss: 0.2415
Epoch 3/7
266/266 ━━━━━━━━━━━━━━━━━━━━ 214s 805ms/step - accuracy: 0.9421 - loss: 0.1809
Epoch 4/7
266/266 ━━━━━━━━━━━━━━━━━━━━ 208s 780ms/step - accuracy: 0.9470 - loss: 0.1596
Epoch 5/7
266/266 ━━━━━━━━━━━━━━━━━━━━ 215s 807ms/step - accuracy: 0.9517 - loss: 0.1491
Epoch 6/7
266/266 ━━━━━━━━━━━━━━━━━━━━ 188s 707ms/step - accuracy: 0.9613 - loss: 0.1231
Epoch 7/7
266/266 ━━━━━━━━━━━━━━━━━━━━ 192s 720ms/step - accuracy: 0.9636 - loss: 0.1057


### saving model outputs as both keras and weights

In [19]:
# save model outputs
os.makedirs(MODEL_SAVE_DIR, exist_ok=True)
save_path = os.path.join(MODEL_SAVE_DIR, f"{MODEL_NAME}.keras") # modern recommended version

model.save(save_path)
print(f"\nModel successfully saved to {save_path}")


Model successfully saved to C:\Users\Shahbaz\Desktop\data science\from git\ku-buildings-classifier\models\cnn_model_phase3_full_data.keras


In [ ]:
final_weights_path = os.path.join(MODEL_SAVE_DIR, f'{MODEL_NAME}_final.weights.h5')
model.save_weights(final_weights_path)
print(f"Final parameters successfully isolated and saved to {final_weights_path}")

Final parameters successfully isolated and saved to C:\Users\Shahbaz\Desktop\data science\from git\ku-buildings-classifier\models\cnn_model_phase3_full_data_final.weights.h5


# Fine-tuning

In [ ]:
basemodel.trainable = True # we set all layers being trainable (removing freeze)

# find out how many layers MobileNetV2 has
total_layers = len(basemodel.layers)
print(f"Total layers of base model: {total_layers}")

# calculate the cutoff point (if we want to tune over only last 30 layers of pretrained model)
fine_tune_at = total_layers - 30

# freeze everything before the cutoff point (again basemodel.trainable = False for first layers till last 30)
for layer in basemodel.layers[:fine_tune_at]:
    layer.trainable = False

print(f"Frozen layers: 0 to {fine_tune_at - 1}")
print(f"Trainable layers (Fine-tuning): {fine_tune_at} to {total_layers - 1}")

Total layers in base model: 154
Frozen layers: 0 to 123
Trainable layers (Fine-tuning): 124 to 153


In [48]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"]
)

In [ ]:
history_full_finetune = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=25,
    callbacks=[early]
)

Epoch 1/25
212/212 ━━━━━━━━━━━━━━━━━━━━ 171s 808ms/step - accuracy: 0.9645 - loss: 0.1113 - val_accuracy: 0.9552 - val_loss: 0.1252
Epoch 2/25
212/212 ━━━━━━━━━━━━━━━━━━━━ 181s 853ms/step - accuracy: 0.9644 - loss: 0.1127 - val_accuracy: 0.9576 - val_loss: 0.1191
Epoch 3/25
212/212 ━━━━━━━━━━━━━━━━━━━━ 180s 847ms/step - accuracy: 0.9617 - loss: 0.1174 - val_accuracy: 0.9576 - val_loss: 0.1204
Epoch 4/25
212/212 ━━━━━━━━━━━━━━━━━━━━ 194s 913ms/step - accuracy: 0.9660 - loss: 0.1095 - val_accuracy: 0.9570 - val_loss: 0.1169
Epoch 5/25
212/212 ━━━━━━━━━━━━━━━━━━━━ 186s 878ms/step - accuracy: 0.9653 - loss: 0.1090 - val_accuracy: 0.9570 - val_loss: 0.1153
Epoch 6/25
212/212 ━━━━━━━━━━━━━━━━━━━━ 198s 931ms/step - accuracy: 0.9672 - loss: 0.1054 - val_accuracy: 0.9582 - val_loss: 0.1176
Epoch 7/25
212/212 ━━━━━━━━━━━━━━━━━━━━ 206s 972ms/step - accuracy: 0.9662 - loss: 0.1142 - val_accuracy: 0.9588 - val_loss: 0.1141
Epoch 8/25
212/212 ━━━━━━━━━━━━━━━━━━━━ 204s 959ms/step - accuracy: 0.9650 -

### saving the fine-tuned model outputs as keras, weights and tflite

In [53]:
# save model outputs
os.makedirs(MODEL_SAVE_DIR, exist_ok=True)
save_path = os.path.join(MODEL_SAVE_DIR, f"fine_tuned.keras") # modern recommended version

model.save(save_path)
print(f"\nModel successfully saved to {save_path}")


Model successfully saved to C:\Users\Shahbaz\Desktop\data science\from git\ku-buildings-classifier\models\fine_tuned.keras


In [54]:
final_weights_path = os.path.join(MODEL_SAVE_DIR, f'fine_tuned_final.weights.h5')
model.save_weights(final_weights_path)
print(f"Final parameters successfully isolated and saved to {final_weights_path}")

Final parameters successfully isolated and saved to C:\Users\Shahbaz\Desktop\data science\from git\ku-buildings-classifier\models\fine_tuned_final.weights.h5


In [ ]:
# tflite conversion
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

tflite_path = os.path.join(MODEL_SAVE_DIR, "fine_tuned_final.tflite")

# save model as tflite
with open(tflite_path, "wb") as f:
    f.write(tflite_model)

## Test the results over 3 models on test data

In [78]:
# set base paths
BASE_DIR = os.getcwd()
MAPPING_PATH = os.path.join(BASE_DIR, "models", "class_mapping.json")
with open(MAPPING_PATH, 'r') as f:
    loaded_class_mapping = {int(k): v for k, v in json.load(f).items()}

test_folders = ['old_test', 'new_test']
models = [
    # 'cnn_model_phase1.keras', # model phase 1
    'cnn_model_phase2.keras', # model phase 2
    'cnn_model_phase3_full_data_final.weights.h5', # model phase 3 (with pretrained model)
    'fine_tuned_final.weights.h5' # with fine-tuned pretrained model
]

all_results = [] # we will collect all outputs

def predict_single_image(image_path, model, class_mapping, model_name):
    # get target size from the loaded model
    img_size = (model.input_shape[1], model.input_shape[2])
    
    img = tf.keras.utils.load_img(image_path, target_size=img_size)
    img_array = tf.keras.utils.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    
    # NORMALIZATION LOGIC:
    if model_name.endswith('.keras'):
        img_array = img_array / 255.0
    
    # Predict
    predictions = model.predict(img_array, verbose=0)
    predicted_index = int(np.argmax(predictions[0]))
    confidence_score = float(np.max(predictions[0]) * 100)
    
    return class_mapping[predicted_index], confidence_score

In [81]:
# We loop over MODELS first, so we don't waste time reloading the same model twice
for model_name in models:
    MODEL_PATH = os.path.join(BASE_DIR, "models", model_name)
    print(f"\n{'='*40}")
    print(f"Loading Model: {model_name}")
    print(f"{'='*40}")
    
    if model_name.endswith('.weights.h5'):
        # Build the empty Phase 3 architecture
        model = tf.keras.Sequential([
            tf.keras.layers.Input(shape=(224,168,3)),
            data_augmentation,
            tf.keras.layers.Rescaling(1./127.5, offset=-1),
            basemodel,
            tf.keras.layers.GlobalAveragePooling2D(),
            
            tf.keras.layers.Dense(256, activation="relu", kernel_initializer="he_normal"),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.4),
            
            tf.keras.layers.Dense(128, activation="relu", kernel_initializer="he_normal"),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.3),
            
            tf.keras.layers.Dense(64, activation="relu", kernel_initializer="he_normal"),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.2),
            
            tf.keras.layers.Dense(32, activation="relu", kernel_initializer="he_normal"),
            tf.keras.layers.Dense(9, activation="softmax") # 9 classes for Phase 3
        ])
        model.load_weights(MODEL_PATH)
        print("Weights loaded successfully!")
        
    else:
        # Load standard Phase 1/2 Keras models
        model = tf.keras.models.load_model(MODEL_PATH)
        print("Keras model loaded successfully!")

    # 3. Loop through test folders for the current model
    for folder_name in test_folders:
        test_folder_path = os.path.join(BASE_DIR, "test_images", folder_name)
        print(f"  -> Scanning folder: {folder_name}...")
        
        for filename in os.listdir(test_folder_path):
            if not filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                continue
            
            full_image_path = os.path.join(test_folder_path, filename)
            
            # Predict
            predicted_building, confidence_score = predict_single_image(full_image_path, model, loaded_class_mapping, model_name)
            
            # Save the result to our dictionary list
            all_results.append({
                "Model Name": model_name,
                "Test Folder": folder_name,
                "Image Name": filename,
                "Predicted Class": predicted_building.upper(),
                "Confidence (%)": round(confidence_score, 2)
            })

# 4. Export to Excel
print("\nCompiling data into Excel...")
df = pd.DataFrame(all_results)
excel_path = os.path.join(BASE_DIR, "detailed_model_outputs.xlsx")
df.to_excel(excel_path, index=False)


Loading Model: cnn_model_phase2.keras
Keras model loaded successfully!
  -> Scanning folder: old_test...
  -> Scanning folder: new_test...

Loading Model: cnn_model_phase3_full_data_final.weights.h5
Weights loaded successfully!
  -> Scanning folder: old_test...
  -> Scanning folder: new_test...

Loading Model: fine_tuned_final.weights.h5
Weights loaded successfully!
  -> Scanning folder: old_test...
  -> Scanning folder: new_test...

Compiling data into Excel...


In [83]:
valid_classes = [name.upper() for name in loaded_class_mapping.values()]

def find_true_class(filename):
    filename_upper = filename.upper()
    for class_name in valid_classes:
        if class_name in filename_upper:
            return class_name
    return "UNKNOWN" # Just in case a file name has no class in it

df['True Class'] = df['Image Name'].apply(find_true_class)
df['Correct'] = df['True Class'] == df['Predicted Class']

accuracy_summary = df.groupby('Model Name')['Correct'].mean() * 100

print("FINAL MODEL ACCURACY REPORT:")
for model_name, accuracy in accuracy_summary.items():
    print(f"{model_name:<45}: {accuracy:>6.2f}%")

analysis_excel_path = os.path.join(BASE_DIR, "model_accuracy_analysis.xlsx")
df.to_excel(analysis_excel_path, index=False)

FINAL MODEL ACCURACY REPORT:
cnn_model_phase2.keras                       :  42.55%
cnn_model_phase3_full_data_final.weights.h5  :  87.23%
fine_tuned_final.weights.h5                  :  93.62%
